# Fetch Data from SDOT

## Step 1: Find data source

To get the data_url,
1. go to the dataset main page, i.e. [Street Permit Dataset](https://data.seattle.gov/Transportation/SDOT-Street-Use-Permit-by-Street-Segment-includes-/w47m-dg37)
2. go to the Export Tab and copy the API Endpoint url.

In [1]:
# data_url = "https://data.seattle.gov/resource/" + ID_DATASET
data_url = "https://data.seattle.gov/resource/w47m-dg37"

## Step 2: Defind query parameters

SODA API use SoQL as query languages, which is very similar to SQL. [Queries using SODA2](http://dev.socrata.com/docs/queries.html) 
  
  
First, we can choose which attributes we want to include in the dateset by seting `$select` in the api call.

In [2]:
# Useful Attributes 
# attributes = ["attr1", "attr2", "attr3"]
# is a array of names of attributes that we want to include in the dataset.
attr_permit = ["permit_status",]
attr_geo = ["shape","shape_length","permit_location_text"]
attr_time = ["am_peak_hour_ok_flag", "pm_peak_hour_ok_flag", "night_weekend_only_flag"]
attr_close = ["sidewalk_closed_flag","sidewalk_close_start_date", "sidewalk_close_end_date"]
attr_block = ["sidewalk_blocked_flag", "sidewalk_block_start_date","sidewalk_block_end_date"]
attr = attr_permit + attr_geo + attr_time + attr_close + attr_block

We can also set querying constraints by setting the parameter `$where`.

In [3]:
# Query Constraints
constraints  = [];

## pick all permits that either block or close the sidewalk.
constraints.append("(sidewalk_closed_flag = 'Y' or sidewalk_blocked_flag = 'Y')")

# pick all blocks/closures happens today.
import datetime
today = datetime.date.today().isoformat()
timeConstraint = "(sidewalk_close_end_date >= '" + today + "' and sidewalk_close_end_date <= '"+ today + "')"
timeConstraint += " or "
timeConstraint += "(sidewalk_block_end_date >= '" + today + "' and sidewalk_block_end_date <= '"+ today + "')"
constraints.append(timeConstraint)

We put all those parameters into a dictionary. The `$limit` parameter indicates maximum number of results to return.

In [4]:
# Create parameter list
params = {"$select": ','.join(attr), 
          "$where": ' and '.join(constraints),
          "$limit": 50000}

## Step3: Request Data

To request data from Socrata, we need a socrata app token. You can get one by  signing up from [this link](https://opendata.socrata.com/login) or use mine.

In [5]:
SOCRATA_KEY = "O196O9J9mXavCQov8jBhT1u75" 
import requests

## get the response from the request
r = requests.get(data_url,
                 params=params,
                 headers={"X-App-Token": SOCRATA_KEY})

assert "message" not in r.json()

/Library/Python/2.7/site-packages/requests/packages/urllib3/util/ssl_.py:90: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning


We can get the dataset from the response.  If there is a AssertionError, it is likely that you make some mistakes in setting the data_url or setting the parameters. You can see the error by print `r.json()["message"]`.

In [6]:
# get the first element in the json file
r.json()[0]

{u'am_peak_hour_ok_flag': u'N',
 u'night_weekend_only_flag': u'N',
 u'permit_location_text': u"104' N C/L OF N 85TH ST. & 24' E OF C/L OF WALLINGFORD AVE N",
 u'permit_status': u'Issued',
 u'pm_peak_hour_ok_flag': u'N',
 u'shape': {u'geometry': {u'paths': [[[-122.3364553319999, 47.69050658400005],
     [-122.33647501999991, 47.69232608000004]]]},
  u'latitude': u'47.69050658400005',
  u'longitude': u'-122.3364553319999',
  u'needs_recoding': False},
 u'shape_length': u'663.708360780565044478862546384334564208984375',
 u'sidewalk_block_end_date': 1435302000,
 u'sidewalk_block_start_date': 1432796400,
 u'sidewalk_blocked_flag': u'Y',
 u'sidewalk_closed_flag': u'N'}

## Step4: Store Data

We can store the json file we get from the request as a local file.

In [7]:
import json
new_file_name = "permits.json"
with open("./" + new_file_name, "w") as f:
    json.dump(r.json(), f)

Then you can find a file named `permits.json` in the local file.

Code for permits:

In [28]:
import datetime
import json
import requests
SOCRATA_KEY = "O196O9J9mXavCQov8jBhT1u75" 


attr_permit = ["permit_status",]
attr_geo = ["shape","shape_length","permit_location_text"]
attr_time = ["am_peak_hour_ok_flag", "pm_peak_hour_ok_flag", "night_weekend_only_flag"]
attr_close = ["sidewalk_closed_flag","sidewalk_close_start_date", "sidewalk_close_end_date"]
attr_block = ["sidewalk_blocked_flag", "sidewalk_block_start_date","sidewalk_block_end_date"]
attr = attr_permit + attr_geo + attr_time + attr_close + attr_block

# Query Constraints

constraints  = []
constraints.append("(sidewalk_closed_flag = 'Y' or sidewalk_blocked_flag = 'Y')")
import datetime
today = datetime.date.today().isoformat()
timeConstraint = "(sidewalk_close_end_date >= '" + today + "' and sidewalk_close_end_date <= '"+ today + "')"
timeConstraint += " or "
timeConstraint += "(sidewalk_block_end_date >= '" + today + "' and sidewalk_block_end_date <= '"+ today + "')"
constraints.append(timeConstraint)

# Create parameter list
params = {"$select": ','.join(attr), 
          "$where": ' and '.join(constraints),
          "$limit": 50000}

data_url = "https://data.seattle.gov/resource/w47m-dg37"
r = requests.get(data_url,
                 params=params,
                 headers={"X-App-Token": SOCRATA_KEY})

json_list = r.json()

with open("./permits.json", "w") as f:
    json.dump(json_list, f)

/Library/Python/2.7/site-packages/requests/packages/urllib3/util/ssl_.py:90: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning


Code for Beacons:


In [49]:
https://data.seattle.gov/resource/iwrq-qjta.json

    import datetime
import json
import requests
SOCRATA_KEY = "O196O9J9mXavCQov8jBhT1u75" 


attr_permit = ["permit_status",]
attr_geo = ["shape","shape_length","permit_location_text"]
attr_time = ["am_peak_hour_ok_flag", "pm_peak_hour_ok_flag", "night_weekend_only_flag"]
attr_close = ["sidewalk_closed_flag","sidewalk_close_start_date", "sidewalk_close_end_date"]
attr_block = ["sidewalk_blocked_flag", "sidewalk_block_start_date","sidewalk_block_end_date"]
attr = attr_permit + attr_geo + attr_time + attr_close + attr_block

# Query Constraints

constraints  = []
constraints.append("(sidewalk_closed_flag = 'Y' or sidewalk_blocked_flag = 'Y')")
import datetime
today = datetime.date.today().isoformat()
timeConstraint = "(sidewalk_close_end_date >= '" + today + "' and sidewalk_close_end_date <= '"+ today + "')"
timeConstraint += " or "
timeConstraint += "(sidewalk_block_end_date >= '" + today + "' and sidewalk_block_end_date <= '"+ today + "')"
constraints.append(timeConstraint)

# Create parameter list
params = {"$select": ','.join(attr), 
          "$where": ' and '.join(constraints),
          "$limit": 50000}


r = requests.get(data_url,
                 params=params,
                 headers={"X-App-Token": SOCRATA_KEY})

json_list = r.json()

with open("./permits.json", "w") as f:
    json.dump(json_list, f)

SyntaxError: invalid syntax (<ipython-input-49-aa80937c9b1d>, line 1)

In [57]:

import json
import requests
SOCRATA_KEY = "O196O9J9mXavCQov8jBhT1u75" 

data_url = "https://data.seattle.gov/resource/aykm-6cyc"
r = requests.get(data_url,
                 params = {},
                 headers={"X-App-Token": SOCRATA_KEY})


/Library/Python/2.7/site-packages/requests/packages/urllib3/util/ssl_.py:90: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning


In [58]:
r

<Response [500]>

In [60]:
r.json()

{u'error': True, u'message': u'Internal error', u'status': 500}